In [ ]:
# !pip install --upgrade pip
# !pip uninstall numpy
# !pip install numpy==1.26.4
# !pip uninstall pandas
# !pip install pandas==2.1.4
# !pip uninstall scikit-learn
# !pip install scikit-learn==1.5.1
# import sklearn
# print(sklearn.__version__)

## **Penyiapan dan Visualisasi Dataset**

In [ ]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets

# load dataset
iris = datasets.load_iris()

# visualisasi dataset
import seaborn as sns

df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['Target'] = pd.Series(iris.target)

print("Data Head: \n", df.head(), "\n")
print("Data Distribution: \n", df.describe(), "\n")
sns.pairplot(df, hue='Target', palette=("tab10"))

## **Splitting, Standarisasi, dan Normalisasi Dataset**

In [ ]:
# splitting dataset into train-set and test-set
from sklearn.model_selection import train_test_split
x = iris.data # feature
y = iris.target # label (3 kategori: 0, 1, 2)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print("x_train: ", x_train.shape)
print("y_test: ", y_test.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)

print("Contoh data test: \n", x_test)

In [ ]:
# standarisasi dataset (pemerataan distribusi)
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(x_train)
# x_train = scaler.transform(x_train)
# x_test = scaler.transform(x_test)
# print("data setelah standarisasi: \n", x_test)

In [ ]:
# normalisasi dataset (mengecilkan/membesarkan jarak antar nilai)
from sklearn.preprocessing import Normalizer

normalizer = Normalizer().fit(x_train)
x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)
print("data setelah normalisasi: \n", x_test)

## **Training Model**

In [ ]:
# untuned model
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train, y_train)

In [ ]:
# tuned model dengan hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {'n_estimators': randint(50,500), 'max_depth': randint(1,20)}

rf = RandomForestClassifier()

# mencari parameter dengan randomized cv
rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, n_iter = 5, cv = 5)

rand_search.fit(x_train, y_train)

best_rf = rand_search.best_estimator_

print('Best hyperparameters:',  rand_search.best_params_)

## **Uji dan Validasi Hasil**

In [ ]:
# validasi dengan test set
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# # validasi untuk untuned model
# y_pred = rf.predict(x_test)

# validasi untuk tuned model
y_pred = best_rf.predict(x_test)

print(classification_report(y_test, y_pred))
print("Akurasi: ", round(accuracy_score(y_test, y_pred) * 100, 2))
print("Presisi: ", round(precision_score(y_test, y_pred, average='weighted') * 100, 2))
print("Recall : ", round(recall_score(y_test, y_pred, average='weighted') * 100, 2))

cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
# validasi dengan k-fold cross-validation
from sklearn.model_selection import KFold, cross_val_score

# # coba cv untuk untuned model
# scores = cross_val_score(rf, x, y, cv=5, scoring='accuracy') * 100

# coba cv untuk tuned model
scores = cross_val_score(best_rf, x, y, cv=5, scoring='accuracy') * 100

print("Accuracy per fold: ", scores)
print("Average accuracy : ", round(scores.mean(), 2), "%")

## **Visualisasi Hasil**

In [ ]:
# visualisasi random forest
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

feature_names = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]
# "sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"

# print("Jumlah decision tree dalam rf: ", len(rf.estimators_), "\n") # untuned model
print("Jumlah decision tree dalam rf: ", len(best_rf.estimators_), "\n") # tuned model

for i in range(3): # hanya tiga decision tree pertama
    # tree = rf.estimators_[i] # untuned model
    tree = best_rf.estimators_[i] # tuned model
    dot_data = export_graphviz(tree, feature_names = feature_names, filled=True, impurity=False, proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
# Create a series containing feature importances from the model and feature names from the training data
# feature_importances = pd.Series(rf.feature_importances_, index=iris.feature_names).sort_values(ascending=False) # untuned model
feature_importances = pd.Series(best_rf.feature_importances_, index=iris.feature_names).sort_values(ascending=False) # tuned model

# Plot a simple bar chart
feature_importances.plot.bar();